<a href="https://colab.research.google.com/github/cabbi120101/final_multi/blob/main/Try/%ED%95%9C%EA%B8%80%EC%9D%84_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC%ED%95%B4%EB%B3%B4%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [2]:
champ_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/동욱코드모음/마지막프로젝트/3주차/시도/머신러닝 부분/champ_kor.csv")
champ_info.head()

,Unnamed: 0,version,id,key,name,title,blurb,info,image,tags,partype,stats,attack,defense,magic,difficulty,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,skins,lore,allytips,enemytips,spells,passive,recommended,skillQ,skillW,skillE,skillR
0,0,11.17.1,Aatrox,266,아트록스,다르킨의 검,한때는 공허에 맞서 싸웠던 슈리마의 명예로운 수호자 아트록스와 그의 종족은 결국 공...,"{'attack': 8, 'defense': 4, 'magic': 3, 'diffi...","{'full': 'Aatrox.png', 'sprite': 'champion0.pn...","['Fighter', 'Tank']",피의 샘,"{'hp': 580, 'hpperlevel': 90, 'mp': 0, 'mpperl...",8,4,3,4,580.0,90,0.0,0.0,345,38,3.25,32.0,1.25,175,3.00,1.00,0.000,0.0,0,0,60.0,5.00,2.500,0.651,"[{'id': '266000', 'num': 0, 'name': 'default',...",한때는 공허에 맞서 싸웠던 슈리마의 명예로운 수호자 아트록스와 그의 종족은 결국 공...,['공격 성공률을 높이려면 다르킨의 검을 사용하는 동안 파멸의 돌진을 사용하세요.'...,"['아트록스의 공격은 미리 표시되므로 공격이 예상되는 지역에서 벗어나세요.', '아...","[{'id': 'AatroxQ', 'name': '다르킨의 검', 'descript...","{'name': '사신 태세', 'description': '주기적으로 아트록스의 ...",[],"{'id': 'AatroxQ', 'name': '다르킨의 검', 'descripti...","{'id': 'AatroxW', 'name': '지옥사슬', 'description...","{'id': 'AatroxE', 'name': '파멸의 돌진', 'descripti...","{'id': 'AatroxR', 'name': '세계의 종결자', 'descript..."
1,1,11.17.1,Ahri,103,아리,구미호,룬테라가 가진 잠재력과 선천적으로 연결된 아리는 마법을 순수한 에너지의 구체로 변환...,"{'attack': 3, 'defense': 4, 'magic': 8, 'diffi...","{'full': 'Ahri.png', 'sprite': 'champion0.png'...","['Mage', 'Assassin']",마나,"{'hp': 526, 'hpperlevel': 92, 'mp': 418, 'mppe...",3,4,8,5,526.0,92,418.0,25.0,330,21,3.50,30.0,0.50,550,5.50,0.60,8.000,0.8,0,0,53.0,3.00,2.000,0.668,"[{'id': '103000', 'num': 0, 'name': 'default',...",룬테라가 가진 잠재력과 선천적으로 연결된 아리는 마법을 순수한 에너지의 구체로 변환...,['스킬을 연속으로 쓸 때 매혹으로 시작하면 현혹의 구슬과 여우불의 적중률이 크게 ...,"['아리는 궁극기 혼령 질주가 없으면 생존력이 극도로 낮아집니다.', '미니언 뒤에...","[{'id': 'AhriOrbofDeception', 'name': '현혹의 구슬'...","{'name': '정기 흡수', 'description': '아리가 9명의 적에게 ...",[],"{'id': 'AhriOrbofDeception', 'name': '현혹의 구슬',...","{'id': 'AhriFoxFire', 'name': '여우불', 'descript...","{'id': 'AhriSeduce', 'name': '매혹', 'descriptio...","{'id': 'AhriTumble', 'name': '혼령 질주', 'descrip..."
2,2,11.17.1,Akali,84,아칼리,섬기는 이 없는 암살자,킨코우 결사단과 그림자의 권이라는 지위를 버린 아칼리는 아이오니아인들에게 필요한 강...,"{'attack': 5, 'defense': 3, 'magic': 8, 'diffi...","{'full': 'Akali.png', 'sprite': 'champion0.png...",['Assassin'],기력,"{'hp': 500, 'hpperlevel': 105, 'mp': 200, 'mpp...",5,3,8,7,500.0,105,200.0,0.0,345,23,3.50,37.0,1.25,125,8.00,0.50,50.000,0.0,0,0,62.0,3.30,3.200,0.625,"[{'id': '84000', 'num': 0, 'name': 'default', ...",킨코우 결사단과 그림자의 권이라는 지위를 버린 아칼리는 아이오니아인들에게 필요한 강...,['아칼리는 체력이 낮은 챔피언을 처치하는 데 뛰어납니다. 아군이 먼저 공격하기를 ...,['아칼리는 황혼의 장막 안에 숨어도 광역 스킬에 영향을 받을 수 있습니다. 이 경...,"[{'id': 'AkaliQ', 'name': '오연투척검', 'descriptio...","{'name': '암살자의 표식', 'description': '스킬 공격으로 챔피...",[],"{'id': 'AkaliQ', 'name': '오연투척검', 'description...","{'id': 'AkaliW', 'name': '황혼의 장막', 'descriptio...","{'id': 'AkaliE', 'name': '표창곡예', 'description'...","{'id': 'AkaliR', 'name': '무결처형', 'description'..."
3,3,11.17.1,Akshan,166,아크샨,떠도는 감시자,"상반신을 드러낸 아크샨은 위험에 직면하면 눈썹을 치켜올리며 당당한 카리스마, 정의로...","{'attack': 0, 'defense': 0, 'magic': 0, 'diffi...","{'full': 'Akshan.png', 'sprite': 'champion0.pn...","['Marksman', 'Assassin']",마나,"{'hp': 560, 'hpperlevel': 90, 'mp': 350, 'mppe...",0,0,0,0,560.0,90,350.0,40.0,330,26,3.00,30.0,0.50,500,3.75,0.65,8.175,0.7,0,0,52.0,3.50,4.000,0.638,"[{'id': '166000', 'num': 0, 'name': 'default',...","상반신을 드러낸 아크샨은 위험에 직면하면 눈썹을 치켜올리며 당당한 카리스마, 정의로...","[""상반신을 드러낸 아크샨은 위험에 직면하면 눈썹을 치켜올리며 당당한 카리스마, 정...",[],"[{'id': 'AkshanQ', 'name': '복수의 부메랑', 'descrip...","{'name': '비열한 싸움', 'description': '아크샨의 기본 공격 ...",[],"{'id': 'AkshanQ', 'name': '복수의 부메랑', 'descript...","{'id': 'AkshanW', 'name': '악당 처단', 'descriptio...","{'id': 'AkshanE', 'name': '영웅의 비상', 'descripti...","{'id': 'AkshanR', 'name': '인과응보', 'description..."
4,4,11.17.1,Alistar,12,알리스타,미노타우로스,비할 데 없이 강력한 전사라는 평판이 자자한 알리스타는 녹서스 제국에게 부족 전체가...,"{'

In [3]:
champ_info[29:30]

,Unnamed: 0,version,id,key,name,title,blurb,info,image,tags,partype,stats,attack,defense,magic,difficulty,hp,hpperlevel,mp,mpperlevel,movespeed,armor,armorperlevel,spellblock,spellblockperlevel,attackrange,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,skins,lore,allytips,enemytips,spells,passive,recommended,skillQ,skillW,skillE,skillR
29,29,11.17.1,Fiddlesticks,9,피들스틱,오래된 공포,룬테라에 무시무시한 고대의 무언가가 깨어났다. 시간을 초월한 공포의 존재 피들스틱은...,"{'attack': 2, 'defense': 3, 'magic': 9, 'diffi...","{'full': 'Fiddlesticks.png', 'sprite': 'champi...","['Mage', 'Support']",마나,"{'hp': 580.4, 'hpperlevel': 92, 'mp': 500, 'mp...",2,3,9,9,580.4,92,500.0,28.0,335,34,3.5,30.0,0.5,480,5.5,0.6,8.0,0.8,0,0,55.36,2.63,2.11,0.625,"[{'id': '9000', 'num': 0, 'name': 'default', '...",룬테라에 무시무시한 고대의 무언가가 깨어났다. 시간을 초월한 공포의 존재 피들스틱은...,[],[],"[{'id': 'FiddleSticksQ', 'name': '공포', 'descri...","{'name': '무해한 허수아비', 'description': '피들스틱의 장신구...",[],"{'id': 'FiddleSticksQ', 'name': '공포', 'descrip...","{'id': 'FiddleSticksW', 'name': '풍작', 'descrip...","{'id': 'FiddleSticksE', 'name': '수확', 'descrip...","{'id': 'FiddleSticksR', 'name': '까마귀 폭풍', 'des..."


In [4]:
from ast import literal_eval

champ_info['skillQ'] = champ_info['skillQ'].apply(literal_eval)
champ_info['skillW'] = champ_info['skillW'].apply(literal_eval)
champ_info['skillE'] = champ_info['skillE'].apply(literal_eval)
champ_info['skillR'] = champ_info['skillR'].apply(literal_eval)

In [5]:
champ_q = []
for i in range(len(champ_info)):
    champ_q.append(champ_info['skillQ'][i]['description'])

In [6]:
champ_w = []
for i in range(len(champ_info)):
    champ_w.append(champ_info['skillW'][i]['description'])

In [7]:
champ_e = []
for i in range(len(champ_info)):
    champ_e.append(champ_info['skillE'][i]['description'])

In [8]:
champ_r = []
for i in range(len(champ_info)):
    champ_r.append(champ_info['skillR'][i]['description'])

In [9]:
champ_summ = champ_info[['id','key','tags']]

In [10]:
champ_summ['champ_q'] = champ_q
champ_summ['champ_w'] = champ_w
champ_summ['champ_e'] = champ_e
champ_summ['champ_r'] = champ_r
champ_summ['champ_all_skill'] = champ_summ['champ_q']+champ_summ['champ_w']+champ_summ['champ_e']+champ_summ['champ_r']

In [11]:
champ_tips = champ_info[['key','allytips','enemytips']]

In [12]:
champ_tips['allytips'] = champ_tips['allytips'].apply(literal_eval)
champ_tips['enemytips'] = champ_tips['enemytips'].apply(literal_eval)

In [13]:
champ_tips['all_tips'] = champ_tips['enemytips'] + champ_tips['allytips']

In [14]:
champ_summ['all_tips'] = champ_tips['all_tips']

In [15]:
champ_summ['all_tips_literal'] = champ_summ['all_tips'].apply(lambda x : (' ').join(x))

In [16]:
champ_summ['all_discribtion'] = champ_summ['all_tips_literal'] + champ_summ['champ_all_skill']

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [18]:
pip install pororo

     |████████████████████████████████| 256 kB 5.4 MB/s 
     |████████████████████████████████| 1.5 MB 37.7 MB/s 
     |████████████████████████████████| 468 kB 48.3 MB/s 
     |████████████████████████████████| 42.4 MB 53 kB/s 
     |████████████████████████████████| 5.9 MB 13.1 MB/s 
     |████████████████████████████████| 748.8 MB 9.8 kB/s 
     |████████████████████████████████| 2.8 MB 26.4 MB/s 
     |████████████████████████████████| 3.1 MB 51.3 MB/s 
     |████████████████████████████████| 1.7 MB 33.3 MB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 145 kB 52.6 MB/s 
     |████████████████████████████████| 90 kB 6.9 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 50 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 57.9 MB/s 
     |████████████████████████████████| 636 kB 51.2 MB/s 
     |███████████████

In [19]:
from pororo import Pororo

[Korean Sentence Splitter]: Initializing Kss...


In [20]:
def make_it_simm(df,column,champ_name):
    count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
    clean_discrib_mat = count_vect.fit_transform(df[column])
    all_discribtion_sim = cosine_similarity(clean_discrib_mat, clean_discrib_mat)
    all_discribtion_sorted_ind = all_discribtion_sim.argsort()[:, ::-1]
    all_discribtion_sim_sorted_ind = np.sort(all_discribtion_sim, axis=1)[:,::-1]
    # titile 데이터 추출
    champ_skill = df[df['id'] == champ_name]
    
    # title_name에 해당되는 데이터프레임 index 객체를 배열로 반환하고
    # 유사도 순으로 top_n개의 index 추출
    champ_index = champ_skill.index.values
    similar_indexes = all_discribtion_sorted_ind[champ_index, :(10)]
    print(all_discribtion_sim_sorted_ind[champ_index, :(10)])
    # 추출된 top_n index 출력 
    # 2차원 데이터를 1차원 array로 변환
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    # index에 해당되는 데이터프레임 반환
    return df.loc[similar_indexes][['id', 'tags']]


In [21]:
make_it_simm(champ_summ, 'all_discribtion', 'DrMundo')

[[1.         0.28964925 0.23284118 0.22905312 0.21819097 0.20818397
  0.20653373 0.20293173 0.20262051 0.1954839 ]]
[[ 24  66  92 102 141  38  39  98 145  13]]


,id,tags
24,DrMundo,"['Fighter', 'Tank']"
66,Lillia,"['Fighter', 'Mage']"
92,Qiyana,"['Assassin', 'Fighter']"
102,Ryze,"['Mage', 'Fighter']"
141,Volibear,"['Fighter', 'Tank']"
38,Gwen,"['Fighter', 'Assassin']"
39,Hecarim,"['Fighter', 'Tank']"
98,Renekton,"['Fighter', 'Tank']"
145,XinZhao,"['Fighter', 'Assassin']"
13,Blitzcrank,"['Tank', 'Fighter']"


## 뽀로로

In [22]:
pip install konlpy   

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
     |████████████████████████████████| 448 kB 61.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [23]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [24]:
## 명사
import nltk
nltk.download('punkt')
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Hannanum

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [25]:
def only_NN(column):

    this_champ_NN = []
    for i in range(len(champ_summ)):
        champ_NN = ' '
        for word in nltk.tag.pos_tag(nltk.tokenize.word_tokenize(champ_summ[column][i])): #명사
            # if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            #     champ_NN = champ_NN + ' ' + word[0]
            if word[1] in ['NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS']: #명사, 동사, 형용사
                champ_NN = champ_NN + ' ' + word[0]
        this_champ_NN.append(champ_NN)
    return this_champ_NN

In [26]:
def really_only_NN(column):

    this_champ_NN = []
    for i in range(len(champ_summ)):
        champ_NN = ' '
        for word in nltk.tag.pos_tag(nltk.tokenize.word_tokenize(champ_summ[column][i])): #명사
            # if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            #     champ_NN = champ_NN + ' ' + word[0]
            if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']: #명사, 동사, 형용사
                champ_NN = champ_NN + ' ' + word[0]
        this_champ_NN.append(champ_NN)
    return this_champ_NN

In [27]:
def only_VB(column):

    this_champ_NN = []
    for i in range(len(champ_summ)):
        champ_NN = ' '
        for word in nltk.tag.pos_tag(nltk.tokenize.word_tokenize(champ_summ[column][i])): 
            # if word[1] in ['NN', 'NNS', 'NNP', 'NNPS']:
            #     champ_NN = champ_NN + ' ' + word[0]
            if word[1] in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS']: # 동사, 형용사
                champ_NN = champ_NN + ' ' + word[0]
        this_champ_NN.append(champ_NN)
    return this_champ_NN

In [28]:
def who_are_u(champ_key):
    return champ_summ['id'][champ_key]

In [29]:
def pororo_good_update(column, champ_name):
    # clean_discrib = []
    cands = []
    for i in range(len(champ_summ[column])):
        cands.append(champ_summ[column][i])
    se = Pororo(task="sentence_embedding", lang="ko")

    key = champ_summ[champ_summ['id'] == champ_name].index.values[0]
    discrib = champ_summ[column][key]
    # 랭킹 매기는 부분
    ranking_1 = se.find_similar_sentences(discrib, cands)['ranking'][:6]
    # 잘 보여주기 부분
    ranking_list = pd.DataFrame()
    champ_rank = []
    how_close = []
    how = []
    for i in range(len(ranking_1)):
        name_1 = who_are_u(ranking_1[i][0])
        champ_rank.append(name_1)
        how_close.append(ranking_1[i][2])
        how.append(ranking_1[i][1])
    ranking_list['champ_rank'] = champ_rank
    ranking_list['how_close'] = how_close
    ranking_list['how'] = how
        
    return ranking_list

In [30]:
champ_summ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                156 non-null    object
 1   key               156 non-null    int64 
 2   tags              156 non-null    object
 3   champ_q           156 non-null    object
 4   champ_w           156 non-null    object
 5   champ_e           156 non-null    object
 6   champ_r           156 non-null    object
 7   champ_all_skill   156 non-null    object
 8   all_tips          156 non-null    object
 9   all_tips_literal  156 non-null    object
 10  all_discribtion   156 non-null    object
dtypes: int64(1), object(10)
memory usage: 13.5+ KB


In [31]:
pororo_good_update('all_discribtion', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나가 일식을 사용하면 피해를 받기 전에 3초 동안 도망칠 시간이 있습니다. 흑점...
1,Pantheon,0.81,판테온이 방패 돌격을 사용하면 정면의 공격으로부터 무적이 됩니다. 따라서 뒤에서 공...
2,Aatrox,0.79,아트록스의 공격은 미리 표시되므로 공격이 예상되는 지역에서 벗어나세요. 아트록스를 ...
3,Gwen,0.79,그웬의 신성한 안개 장막은 그웬을 한 번만 따라오며 이후 그웬이 이동하면 사라집니다...
4,Fizz,0.78,피즈가 강화된 스킬을 사용한 후에는 몇 초 동안 기본 공격이 강력해지니 삼지창이 빛...


In [32]:
pororo_good_update('all_discribtion', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,강철 폭풍의 폭은 아주 좁습니다. 가능하면 옆으로 피하세요. 야스오가 강철 폭풍을 ...
1,Zilean,0.78,질리언의 이동 속도에 맞출 수 있다면 그의 궁극기가 끝날 때까지 기다렸다가 결정타를...
2,Viego,0.75,비에고의 유령 칼날이 적중 시 현재 체력에 비례한 추가 피해를 입히고 최근 비에고의...
3,Lissandra,0.73,"리산드라가 얼음갈퀴 길을 활용해 이동하지 못하도록 막으려면, 이 스킬을 재시전하기 ..."
4,Katarina,0.73,카타리나는 마법 피해를 주로 입히므로 카타리나가 공격력을 올려주는 아이템을 가지고 ...


In [33]:
clean_all_skill = only_NN('champ_all_skill')
champ_summ['clean_all_skill'] = clean_all_skill
pororo_good_update('clean_all_skill', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나가 다음 기본 공격에서 방패를 사용하여 대상에게 추가 마법 피해를 입히고 기절...
1,Darius,0.76,다리우스가 도끼를 큰 원의 형태로 휘두릅니다 도끼날에 맞은 적들은 도끼자루에 맞은 ...
2,Alistar,0.75,알리스타가 땅을 내리쳐 주변의 적에게 피해를 입히고 공중에 띄웁니다.알리스타가 적에...
3,Viktor,0.72,빅토르가 대상에게 강력한 에너지를 주입해 마법 피해를 주고 보호막이 생기면서 다음번...
4,Jinx,0.71,징크스는 '빵야빵야 미니건과 '생선대가리 로켓 런처 중에서 원하는 것을 선택하여 기...


## 우선 nltk로 전처리 한 방법

In [34]:
## 명사 , 동사, 형용사
NN_skill = only_NN('champ_all_skill')
champ_summ['NN_skill'] = NN_skill
pororo_good_update('NN_skill', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나가 다음 기본 공격에서 방패를 사용하여 대상에게 추가 마법 피해를 입히고 기절...
1,Darius,0.76,다리우스가 도끼를 큰 원의 형태로 휘두릅니다 도끼날에 맞은 적들은 도끼자루에 맞은 ...
2,Alistar,0.75,알리스타가 땅을 내리쳐 주변의 적에게 피해를 입히고 공중에 띄웁니다.알리스타가 적에...
3,Viktor,0.72,빅토르가 대상에게 강력한 에너지를 주입해 마법 피해를 주고 보호막이 생기면서 다음번...
4,Jinx,0.71,징크스는 '빵야빵야 미니건과 '생선대가리 로켓 런처 중에서 원하는 것을 선택하여 기...


In [35]:
pororo_good_update('NN_skill', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,전방으로 내질러 일직선 상의 모든 적에게 피해를 입힙니다.적중 시 몇 초간 폭풍 구...
1,Yone,0.82,전방으로 검을 내질러 일직선상의 모든 적에게 피해를 입힙니다.적중 시 몇 초간 폭풍...
2,Jayce,0.81,해머 사용 시 공중으로 도약해 적들에게 물리 피해를 입히고 이동 속도를 감소시킵니다...
3,Jhin,0.80,진이 마법 폭탄을 적에게 던집니다 최대 네 명의 대상에게 적중하며 적을 처치할 때마...
4,Xerath,0.78,긴 사거리를 지닌 에너지 포를 쏘아 적중한 모든 적에게 마법 피해를 줍니다.비전 에...


In [36]:
# 진짜 명사만 돌림
real_NN_skill = really_only_NN('champ_all_skill')
champ_summ['real_NN_skill'] = real_NN_skill
pororo_good_update('real_NN_skill', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,다음 기본 공격에서 방패를 사용하여 대상에게 추가 마법 피해를 입히고 기절시킵니다....
1,Darius,0.79,도끼를 큰 원의 형태로 휘두릅니다 적들은 도끼자루에 맞은 적들보다 더 큰 피해를 입...
2,Viktor,0.76,대상에게 강력한 에너지를 주입해 마법 피해를 주고 보호막이 생기면서 다음번 기본 공...
3,Alistar,0.75,땅을 내리쳐 주변의 적에게 피해를 입히고 공중에 띄웁니다.알리스타가 적에게 박치기를...
4,Rammus,0.74,공 모양으로 적을 향해 굴러가며 충돌 시 피해를 입히고 적의 이동 속도를 늦춥니다....


In [37]:
pororo_good_update('real_NN_skill', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,내질러 일직선 상의 모든 적에게 피해를 입힙니다.적중 시 몇 초간 폭풍 구름 중첩됩...
1,Jayce,0.81,사용 시 도약해 적들에게 물리 피해를 입히고 이동 속도를 감소시킵니다.캐논 사용 시...
2,Yone,0.79,검을 내질러 일직선상의 모든 적에게 피해를 입힙니다.적중 시 몇 초간 폭풍 구름 중...
3,Kled,0.79,밧줄에 묶인 덫을 던져 적 챔피언에게 피해를 입히고 붙잡습니다 잠시 동안 근거리를 ...
4,Kayle,0.76,차원문을 열어 적을 관통하는 천상의 검을 소환합니다 모든 대상에게 피해를 입히고 이...


In [38]:
# 동사 형용사
VB_skill = only_VB('champ_all_skill')
champ_summ['VB_skill'] = VB_skill
pororo_good_update('VB_skill', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나가 지속시간이 끝날 이 형상이 이 범위
1,Galio,0.65,갈리오가 스킬을 다시 잠시 후
2,Yone,0.53,전방으로 효과가 중첩되면 적중한 챔피언 지속시간이 끝나면
3,Amumu,0.53,아무무가 활성화 시 아무무가 공격당할 대기시간이
4,Leblanc,0.52,르블랑이 주고 표식이 남은 안에 동안


In [39]:
pororo_good_update('VB_skill', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,전방으로 효과가 중첩되면 사용할 때마다 기류가 최대치로
1,Yone,0.62,전방으로 효과가 중첩되면 적중한 챔피언 지속시간이 끝나면
2,Taric,0.58,충전량을 타릭이 스킬을
3,Kennen,0.55,케넨이 스킬 활성화시 번개로 변신하면
4,Rengar,0.53,렝가의 피해량과 모든 대상을 사냥의 전율이


In [40]:
## 명사 , 동사, 형용사
NN_discrib = only_NN('all_discribtion')
champ_summ['NN_discrib'] = NN_discrib
pororo_good_update('NN_discrib', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나가 일식을 사용하면 피해를 받기 전에 동안 도망칠 시간이 있습니다 흑점 폭발의...
1,Pantheon,0.81,판테온이 방패 돌격을 사용하면 정면의 공격으로부터 무적이 됩니다 따라서 뒤에서 공격...
2,Gwen,0.80,그웬의 신성한 안개 장막은 그웬을 한 번만 따라오며 이후 그웬이 이동하면 사라집니다...
3,Illaoi,0.79,일라오이를 상대할 때는 수시로 촉수를 처치하는 것이 승리의 발판이 됩니다 자신의 영...
4,Aatrox,0.78,아트록스의 공격은 미리 표시되므로 공격이 예상되는 지역에서 벗어나세요 아트록스를 향...


In [41]:
pororo_good_update('NN_discrib', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,강철 폭풍의 폭은 아주 좁습니다 가능하면 옆으로 피하세요 야스오가 강철 폭풍을 연이...
1,Lissandra,0.84,리산드라가 얼음갈퀴 길을 활용해 이동하지 못하도록 막으려면 이 스킬을 재시전하기 전...
2,Varus,0.81,역병에 감염된 경우 바루스의 스킬 공격으로 추가 피해를 입을 수 있습니다 적을 처치...
3,Katarina,0.81,카타리나는 마법 피해를 주로 입히므로 카타리나가 공격력을 올려주는 아이템을 가지고 ...
4,Jhin,0.81,살상연희 스킬은 최근 안에 진의 기본 공격이나 함정 또는 진의 아군에게 공격당한 적...


In [42]:
# 진짜 명사만 돌림
real_NN_discrib = really_only_NN('all_discribtion')
champ_summ['real_NN_discrib'] = real_NN_discrib
pororo_good_update('real_NN_discrib', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,일식을 사용하면 피해를 받기 동안 도망칠 시간이 있습니다 한가운데 있는 적만 기절하...
1,Gwen,0.83,신성한 안개 장막은 그웬을 한 번만 따라오며 이후 그웬이 이동하면 사라집니다 적중시...
2,Akali,0.82,황혼의 장막 안에 숨어도 광역 스킬에 영향을 받을 수 있습니다 잠시 동안 아칼리의 ...
3,LeeSin,0.82,신의 궁극기인 용의 분노로 받는 피해를 최소화하려면 유닛을 분산시키십시오 강철의 의...
4,Katarina,0.82,마법 피해를 주로 입히므로 카타리나가 공격력을 올려주는 아이템을 가지고 있다 하더라...


In [43]:
pororo_good_update('real_NN_discrib', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,폭풍의 폭은 아주 좁습니다 피하세요 폭풍을 연이어 두 번 맞히면 다음에는 회오리가 ...
1,Caitlyn,0.86,필트오버 피스메이커를 사용하여 견제를 시도한다면 완전히 피하거나 아군 미니언 뒤에 ...
2,Jinx,0.84,미니건은 공격 속도가 상승할 때까지 시간이 걸립니다 로켓으로 공격한다면 덮쳐서 빠르...
3,Heimerdinger,0.84,포탑은 하나씩 제거하기보다 미니언의 도움을 받아 한꺼번에 제거하는 편이 낫습니다 곤...
4,Braum,0.82,동상 스킬 혹은 기본 공격을 맞혀야만 뇌진탕 펀치 중첩을 시작할 수 있습니다 표식이...


In [44]:
# 동사 형용사
VB_discrib = only_VB('all_discribtion')
champ_summ['VB_discrib'] = VB_discrib
pororo_good_update('VB_discrib', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나가 전에 흑점 폭발의 최전선에서 목표 여명의 방패와 일식을 사용하여 지속시간이...
1,Nocturne,0.66,피해망상이 녹턴의 말할 피해망상 스킬은 황혼의 인도자는 전투 시 어둠의 장막을 적이...
2,Syndra,0.64,신드라를 적군 와해는 신드라가 이 신드라의 스킬은 궁극기의 피해량을 의지의 힘으로 ...
3,Pantheon,0.63,판테온이 따라서 뒤에서 체력이 낮은 혜성의 창은 판테온이 거대 이때 멀리 필멸자의 ...
4,Veigar,0.62,암흑 주의를 기울여 사건의 지평선 스킬 영역 베이가의 궁극기는 사건의 지평선 마나 ...


In [45]:
pororo_good_update('VB_discrib', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,강철 가능하면 옆으로 야스오가 강철 해당 스킬의 야스오는 회오리 이 때를 야스오의 ...
1,Lissandra,0.68,리산드라가 리산드라의 서릿발 재사용 대기가 리산드라의 얼음 아군 미니언 자기 자신에...
2,Jayce,0.64,제이스는 제이스의 자세와 가속 관문을 제이스는 게임 제이스가 추가 무기를 되도록 무...
3,LeeSin,0.62,리 리 신은 리 신은 방해 스킬을 용의 분노를 스킬을 시전하는 적에게 줄 자신에게 ...
4,Nautilus,0.62,노틸러스가 미리 달아나면 노틸러스는 보호막이 시간이 있으면 기습할 때 역조 스킬은 ...


## KoNLPy 를 이용해 보자

In [46]:
pip install konlpy

In [47]:
from konlpy.tag import Mecab

In [48]:
kkma = Kkma()

In [49]:
pprint(kkma.nouns(u'질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [50]:
pprint(kkma.pos(u'오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))

[('오류', 'NNG'),
 ('보고', 'NNG'),
 ('는', 'JX'),
 ('실행', 'NNG'),
 ('환경', 'NNG'),
 (',', 'SP'),
 ('에러', 'NNG'),
 ('메세지', 'NNG'),
 ('와', 'JKM'),
 ('함께', 'MAG'),
 ('설명', 'NNG'),
 ('을', 'JKO'),
 ('최대한', 'NNG'),
 ('상세히', 'MAG'),
 ('!', 'SF'),
 ('^^', 'EMO')]


In [51]:
def ko_only_NN(column):

    this_champ_NN = []
    for i in range(len(champ_summ)):

        champ_NN = kkma.nouns(champ_summ[column][i])

        this_champ_NN.append(champ_NN)
    return this_champ_NN

In [52]:
def pororo_good_update(column, champ_name):
    # clean_discrib = []
    cands = []
    for i in range(len(champ_summ[column])):
        cands.append(champ_summ[column][i])
    se = Pororo(task="sentence_embedding", lang="ko")

    key = champ_summ[champ_summ['id'] == champ_name].index.values[0]
    discrib = champ_summ[column][key]
    # 랭킹 매기는 부분
    ranking_1 = se.find_similar_sentences(discrib, cands)['ranking'][:6]
    # 잘 보여주기 부분
    ranking_list = pd.DataFrame()
    champ_rank = []
    how_close = []
    how = []
    for i in range(len(ranking_1)):
        name_1 = who_are_u(ranking_1[i][0])
        champ_rank.append(name_1)
        how_close.append(ranking_1[i][2])
        how.append(ranking_1[i][1])
    ranking_list['champ_rank'] = champ_rank
    ranking_list['how_close'] = how_close
    ranking_list['how'] = how
        
    return ranking_list

In [53]:
## 명사
ko_NN_discrib = ko_only_NN('all_discribtion')
champ_summ['NN_discrib'] = ko_NN_discrib

In [54]:
champ_summ['NN_discrib']

0      [아트, 아트록스, 록스, 공격, 표시, 예상, 지역, 이동, 양옆, 지옥, 지옥사...
1      [아리, 궁극, 궁극기, 기, 혼령, 질주, 생존력, 극도, 미니, 미니언, 언, ...
2      [칼리, 황혼, 장막, 안, 광역, 스킬, 영향, 수, 이, 경우, 잠시, 동안, ...
3      [상반신, 아크, 은, 위험, 직면, 눈썹, 당, 카리스마, 정의, 복수심, 악과,...
4      [알, 알리스타, 리, 스타, 방해, 효과, 스킬, 방어력, 공격수, 목표, 포탑,...
                             ...                        
151    [제, 공격력, 아이템, 구입, 때, 추가, 효과, 때문, 상대, 방어력, 쪽, 효...
152    [직스, 설치, 지뢰, 속도, 스킬, 적중, 재사, 대기, 대기시간, 시간, 편, ...
153    [질, 질리언, 리, 언, 이동, 속도, 수, 그, 궁극, 궁극기가, 기가, 때, ...
154    [별은, 총, 이동, 거리, 비례, 피해, 차원, 후, 원래, 자리, 반격, 때, ...
155    [자, 씨앗, 파괴, 수, 때, 자, 식물, 마지막, 순간, 발, 스킬, 낭비, 곳...
Name: NN_discrib, Length: 156, dtype: object

In [55]:
champ_summ['NN_discrib'] = champ_summ['NN_discrib'].apply(lambda x : (' ').join(x))

In [56]:
pororo_good_update('NN_discrib', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레 레오나 오나 일식 사용 피해 전 3 3초 초 동안 시간 흑점 폭발 가운데 적 기...
1,LeeSin,0.80,리 신 궁극 궁극기인 기인 용의 분노 피해 최소화 유닛 분산 신은 강철 의지 무력화...
2,XinZhao,0.78,신 돌격 기술 주변 유닛 피해 궁극 궁극기가 기가 선제 공격 오 때 진형 최대한 편...
3,Akali,0.78,칼리 황혼 장막 안 광역 스킬 영향 수 이 경우 잠시 동안 위치 오연 오연투척 투척...
4,Ekko,0.76,에코 궁극 궁극기가 기가 사용 대기 뒤 자취 시공간 붕괴 수 판단 기 기절 절 지대...


In [57]:
pororo_good_update('NN_discrib', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,강철 폭풍 폭 가능 옆 야스오 번 다음 회오리 발사 당 스킬 음향 효과 수 준비 바...
1,Varus,0.85,역병 감염 경우 바 바루스 루스 스킬 공격 추가 피해 수 적 처치 어시스트 잠시 속...
2,Quinn,0.84,발 발러 러 표식 퀸 이득 수 후퇴 현재 위치 예의 예의주시 주시 후방 지원 스킬 ...
3,Kled,0.83,빵 빵야 야 기본 공격 적 피해 미니 미니언 언 처치 구조물 몬스터 스칼 용기 회복...
4,Vi,0.83,금고 수기 충전 피해 피해량 량 배 후퇴 공격 번 연속 방어구 바 바이의 이의 속도...


In [58]:
## 명사
ko_NN_skill = ko_only_NN('champ_all_skill')
champ_summ['NN_skill'] = ko_NN_skill
champ_summ['NN_skill'] = champ_summ['NN_skill'].apply(lambda x : (' ').join(x))

In [59]:
pororo_good_update('NN_skill', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레 레오나 오나 다음 기본 공격 방패 사용 대상 추가 마법 피해 기절 피해량 량 감...
1,Irelia,0.76,렐 렐리아가 리 아가 돌진 대상 공격 체력 회복 표식 칼날 쇄도 적 처치 사용 대기...
2,Galio,0.74,리 리오가 오가 개 돌풍 발사 지속 피해 거대 소용돌이 방어 태세 움직임 스킬 사용...
3,Fiora,0.73,피 피오라 오라 방향 돌진 근처 적 하나 물리 피해 적중 시 효과 적용 잠시 동안 ...
4,Velkoz,0.73,벨코즈 플라즈마 광선 발사 나 스킬 사용 적 둘 적중 시 둔화 적용 피해 벨 벨코즈...


In [60]:
pororo_good_update('NN_skill', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,전방 일직선 상의 적 피해 적중 시 초간 폭풍 구름 효과 1 1회 회 중첩 2 2회...
1,Yone,0.86,전방 검 일직선 적 피해 적중 시 초간 폭풍 구름 효과 1 1회 회 중첩 2 2회 ...
2,Jayce,0.80,해머 사용 시 공중 도약 적 물리 피해 이동 속도 감소 캐논 사거리 끝 폭발 전기 ...
3,Sett,0.78,세트 다음 번 공격 대상 최대 체력 비례 추가 피해 적 챔피언 이동 때 속도 기본 ...
4,Janna,0.78,잔 나 기압 온도 국지적 변경 시간 폭풍 생성 스킬 사용 수 방향 경로 적 피해 공...


## KoNLPy 를 이용해 보자 2번 째

In [61]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='/tmp/dic.txt')

In [62]:
print(komoran.nouns(u'오픈소스에 관심 많은 멋진 개발자님들!'))

['오픈', '소스', '관심', '개발자']


In [63]:
def komoran_only_NN(column):

    this_champ_NN = []
    for i in range(len(champ_summ)):

        champ_NN = komoran.nouns(champ_summ[column][i])

        this_champ_NN.append(champ_NN)
    return this_champ_NN

In [64]:
## 명사
komoran_NN_skill = komoran_only_NN('champ_all_skill')
champ_summ['NN_skill_komoran'] = komoran_NN_skill
champ_summ['NN_skill_komoran'] = champ_summ['NN_skill_komoran'].apply(lambda x : (' ').join(x))

In [65]:
pororo_good_update('NN_skill_komoran', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오 나가 다음 기본 공격 방패 사용 대상 추가 마법 피해 기절 레오 나가 방패 피...
1,Morgana,0.75,흑마 법 적 제자리 속박 적 유발 고통 마법 피해 대상 지역 저주 그림자 원 안 적...
2,Veigar,0.75,베이 가가 어둠 에너지 발사 처음 적 마법 피해 스키 적 유닛 처치 베이 가의 주문...
3,Rell,0.74,레 이 전방 창 일직선 유닛 마법 피해 보호막 파괴 처음 창 유닛 갑옷 파쇄 방어력...
4,Darius,0.74,다리우스 도끼 원 형태 도끼날 적 도끼 자루 적 피해 다리우스 도끼날 적 챔피언 대...


In [66]:
pororo_good_update('NN_skill_komoran', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,전방 일직선 상 적 피해 적중 시 초간 폭풍 구름 효과 회 중첩 회 중첩 강철 폭풍...
1,Graves,0.83,탄환 발사 탄환 발사 초 뒤 지형 충돌 시 초 뒤 목표 지점 연막탄 발사 시야 연기...
2,Jayce,0.82,해머 사용 시 공중 도약 적 물리 피해 이동 속도 감소 캐논 사용 시 적 사거리 끝...
3,Warwick,0.82,앞 도약 대상 대상 최대 체력 비례 피해 이때 피해 량 체력 회복 체력 미만 적 감...
4,Brand,0.81,브랜드 덩이 발사 마법 피해 대상 스키 대상 1.5 초 동안 기절 잠시 후 목표 지...


## 뽀로로를 이용하여 전처리 

In [67]:
pip install python-mecab-ko

     |████████████████████████████████| 200 kB 5.3 MB/s 
  ERROR: Failed building wheel for python-mecab-ko
  Running setup.py clean for python-mecab-ko
Failed to build python-mecab-ko
    Running setup.py install for python-mecab-ko ... done
  DEPRECATION: python-mecab-ko was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [68]:
 pos = Pororo(task="pos", lang="ko")

In [69]:
pos("안녕하세요. 제 이름은 카터입니다.")

[('안녕', 'NNG'),
 ('하', 'XSV'),
 ('시', 'EP'),
 ('어요', 'EF'),
 ('.', 'SF'),
 (' ', 'SPACE'),
 ('저', 'NP'),
 ('의', 'JKG'),
 (' ', 'SPACE'),
 ('이름', 'NNG'),
 ('은', 'JX'),
 (' ', 'SPACE'),
 ('카터', 'NNP'),
 ('이', 'VCP'),
 ('ᄇ니다', 'EF'),
 ('.', 'SF')]

In [70]:
def pororo_good_update(column, champ_name):
    # clean_discrib = []
    cands = []
    for i in range(len(champ_summ[column])):
        cands.append(champ_summ[column][i])
    se = Pororo(task="sentence_embedding", lang="ko")

    key = champ_summ[champ_summ['id'] == champ_name].index.values[0]
    discrib = champ_summ[column][key]
    # 랭킹 매기는 부분
    ranking_1 = se.find_similar_sentences(discrib, cands)['ranking'][:6]
    # 잘 보여주기 부분
    ranking_list = pd.DataFrame()
    champ_rank = []
    how_close = []
    how = []
    for i in range(len(ranking_1)):
        name_1 = who_are_u(ranking_1[i][0])
        champ_rank.append(name_1)
        how_close.append(ranking_1[i][2])
        how.append(ranking_1[i][1])
    ranking_list['champ_rank'] = champ_rank
    ranking_list['how_close'] = how_close
    ranking_list['how'] = how
        
    return ranking_list

In [71]:
def poro_NN(column):
    pos = Pororo(task="pos", lang="ko")
    this_champ_NN = []
    for i in range(len(champ_summ)):
        champ_NN = ' '
        for word in pos(champ_summ[column][i]): #명사

            if word[1] in ['NN', 'NNG','NNS', 'NNP', 'NNPS']: #명사, 동사, 형용사
                champ_NN = champ_NN + ' ' + word[0]
        this_champ_NN.append(champ_NN)
    return this_champ_NN

In [72]:
champ_summ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                156 non-null    object
 1   key               156 non-null    int64 
 2   tags              156 non-null    object
 3   champ_q           156 non-null    object
 4   champ_w           156 non-null    object
 5   champ_e           156 non-null    object
 6   champ_r           156 non-null    object
 7   champ_all_skill   156 non-null    object
 8   all_tips          156 non-null    object
 9   all_tips_literal  156 non-null    object
 10  all_discribtion   156 non-null    object
 11  clean_all_skill   156 non-null    object
 12  NN_skill          156 non-null    object
 13  real_NN_skill     156 non-null    object
 14  VB_skill          156 non-null    object
 15  NN_discrib        156 non-null    object
 16  real_NN_discrib   156 non-null    object
 17  VB_discrib      

In [73]:
poro_discrib = poro_NN('all_discribtion')
champ_summ['poro_discrib'] = poro_discrib

In [74]:
pororo_good_update('poro_discrib', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나 일식 사용 피해 전 동안 도망 시간 흑점 폭발 가운데 기절 이동 최 전선 목...
1,Irelia,0.80,렐리아 상대 때 이오니아 열정 중첩 주의 중첩 때 교전 미니언 무리 렐리아 이동 미...
2,Fiora,0.79,치명 검무 오라 공격 곳 때 반격 준비 피 오라 이동 불가 효과 때 주의 응수 스킬...
3,LeeSin,0.79,리 신 궁극 기인 분노 피해 최소 유닛 분산 리 신 강철 의지 무력 스킬 물리 피해...
4,Yone,0.79,전방 검 일 직선 적 피해 입 힙 적중 시 초간 폭풍 구름 효과 중첩 중첩 전방 돌...


In [75]:
pororo_good_update('poro_discrib', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,강철 폭풍 폭 가능 옆 야스오 강철 폭풍 다음 회오리 발사 해당 스킬 음향 효과 준...
1,Varus,0.86,역병 감염 경우 바루 스킬 공격 추가 피해 처치 어시스트 바루 공격 속도 상승 위협...
2,Jayce,0.84,제이스 근접 원 거리 공격 가능 제이스 자세 무기 색깔 근접 원 거리 파악 가속 관...
3,Shyvana,0.83,쉬바 위력 공격 운영 원 거리 공격 수 챔피언 상대 스킬 용 형상 때 적 피해 상대...
4,Kennen,0.81,폭풍 표식 때 조심 상대 폭풍 표식 중첩 기절 기본 체력 번개 질주 생각 사용 때 ...


In [76]:
poro_skill = poro_NN('champ_all_skill')
champ_summ['poro_skill'] = poro_skill

In [77]:
pororo_good_update('poro_skill', 'Leona')

,champ_rank,how_close,how
0,Leona,1.00,레오나 다음 기본 공격 방패 사용 대상 추가 마법 피해 기절 레오나 방패 피해 량 ...
1,Fiora,0.76,오라 방향 돌진 근처 물리 피해 적중 시 효과 적용 피 오라 동안 공격 이동 불가 ...
2,Irelia,0.74,렐리아 돌진 대상 공격 체력 회복 대상 표식 칼 날 쇄도 처치 칼 날 쇄도 사용 대...
3,Sona,0.73,소나 용맹 찬가 연주 선율 발사 주변 적 마법 피해 입 힙 이 때 챔피언 몬스터 공...
4,Viktor,0.72,빅토르 대상 에너지 주입 마법 피해 보호 막 다음 기본 공격 강화 증강 사용 빅토르...


In [78]:
pororo_good_update('poro_skill', 'Yasuo')

,champ_rank,how_close,how
0,Yasuo,1.00,전방 일 직선 상 적 피해 입 힙 적중 시 초간 폭풍 구름 효과 중첩 중첩 강철 폭...
1,Yone,0.83,전방 검 일 직선 적 피해 입 힙 적중 시 초간 폭풍 구름 효과 중첩 중첩 전방 돌...
2,Jayce,0.82,해머 사용 공중 도약 적 물리 피해 이동 속도 감소 사용 적 사 거리 끝 폭발 전기...
3,Sylas,0.82,사일러스 사슬 교차 대상 지점 피해 이동 속도 늦 후 사슬 교차 지점 마력 폭발 피...
4,Varus,0.82,스가 조준한 다음 강화 화살 발사 조준한 시간 비례 사 거리 피해 량 기본 지속 효...
